# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [14]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import datetime

In [2]:
%run create_tables.py

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
pwd

'/Users/juliansanjuan/Documents/Udacity/Project_1'

In [6]:
song_files = get_files('/Users/juliansanjuan/Documents/Udacity/Project_1/data/song_data')

In [7]:
colnames =['num_songs','artist_id','artist_latitude','artist_longitude','artist_location','artist_name','song_id','title','duration','year']
songDict = {
    'num_songs':[],
    'artist_id':[],
    'artist_latitude':[],
    'artist_longitude':[],
    'artist_location':[],
    'artist_name':[],
    'song_id':[],
    'title':[],
    'title':[],
    'duration':[],
    'year':[]
}
for i in range(len(song_files)):
    sData = pd.read_json(song_files[i],typ='Frame',orient='None')
    for i in range(len(colnames)):
        songDict[colnames[i]].append(sData[i])

In [8]:
song_df = pd.DataFrame(songDict)
song_df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
1,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
2,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
3,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
4,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [20]:
song_data = song_df[['song_id','title','artist_id','year','duration']].copy()
song_data = song_data.values
song_data = list(song_data)
song_data

[array(['SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C',
        1982, 233.40363], dtype=object),
 array(['SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984,
        269.81832], dtype=object),
 array(['SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007,
        209.60608], dtype=object),
 array(['SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0,
        189.57016], dtype=object),
 array(['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969,
        148.03546], dtype=object),
 array(['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0,
        218.93179], dtype=object),
 array(['SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', 0,
        177.47546], dtype=object),
 array(['SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)',
        'ARNTLGG11E2835DDB9', 0, 266.39628], dtype=object),
 array(['SOYMRWW12A6D4FAB14',
        'The Moon And I (Ordinary Day Album Version)',
        'ARKFYS91187B98E58F

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [23]:
for i in range(len(song_data)):
    cur.execute(song_table_insert, song_data[i])
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [25]:
artist_data = song_df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].copy()
artist_data = artist_data.values
artist_data = list(artist_data)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [28]:
for i in range(len(artist_data)):    
    cur.execute(artist_table_insert, artist_data[i])
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [29]:
log_files = get_files('/Users/juliansanjuan/Documents/Udacity/Project_1/data/log_data')

In [30]:
for i in range(len(log_files)):
    log_df = pd.read_json(log_files[i],lines=True)

In [31]:
log_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382 entries, 0 to 381
Data columns (total 18 columns):
artist           319 non-null object
auth             382 non-null object
firstName        370 non-null object
gender           370 non-null object
itemInSession    382 non-null int64
lastName         370 non-null object
length           319 non-null float64
level            382 non-null object
location         370 non-null object
method           382 non-null object
page             382 non-null object
registration     370 non-null float64
sessionId        382 non-null int64
song             319 non-null object
status           382 non-null int64
ts               382 non-null int64
userAgent        370 non-null object
userId           382 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 53.8+ KB


In [32]:
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Mudhoney,Logged In,Aleena,F,10,Kirby,231.57506,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Get Into Yours,200,1542326457796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
1,Carpenters,Logged In,Aleena,F,11,Kirby,238.39302,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Yesterday Once More,200,1542326688796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
2,None,Logged In,Aleena,F,12,Kirby,NaN,paid,"Waterloo-Cedar Falls, IA",GET,Home,1.541023e+12,637,None,200,1542326691796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
3,Kings Of Leon,Logged In,Aleena,F,13,Kirby,185.28608,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Taper Jean Girl,200,1542326926796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
4,The Bloody Beetroots,Logged In,Aleena,F,14,Kirby,201.97832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Warp 1.9 (feat. Steve Aoki),200,1542327111796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [33]:
df = log_df[log_df.page == 'NextSong'].copy()
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Mudhoney,Logged In,Aleena,F,10,Kirby,231.57506,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Get Into Yours,200,1542326457796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
1,Carpenters,Logged In,Aleena,F,11,Kirby,238.39302,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Yesterday Once More,200,1542326688796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
3,Kings Of Leon,Logged In,Aleena,F,13,Kirby,185.28608,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Taper Jean Girl,200,1542326926796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
4,The Bloody Beetroots,Logged In,Aleena,F,14,Kirby,201.97832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Warp 1.9 (feat. Steve Aoki),200,1542327111796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
5,Kings Of Leon,Logged In,Aleena,F,15,Kirby,201.79546,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,637,Revelry,200,1542327312796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44


In [34]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-16 00:00:57.796
1   2018-11-16 00:04:48.796
3   2018-11-16 00:08:46.796
4   2018-11-16 00:11:51.796
5   2018-11-16 00:15:12.796
Name: ts, dtype: datetime64[ns]

In [35]:
timestamp = df['ts']
weekday = t.dt.weekday
year = t.dt.year
month = t.dt.month
day = t.dt.day 
hour = t.dt.hour
week = t.dt.week
time_dict = {
            'start_time':timestamp,
            'hour':hour,
            'day': day,
            'week': week,
            'month': month,
            'year': year,
            'weekday': weekday}
    


In [43]:
time_df = pd.DataFrame(time_dict)
time_df.drop_duplicates(inplace=True)
#time_df = time_df.values
#time_df = list(time_df)

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [46]:
for i, row in time_df.iterrows():
#    print(list(row))
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [49]:
user_df = df[['userId','firstName','lastName','gender','level']].copy()
user_df.drop_duplicates(inplace=True)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [50]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.